# Many to One Explainer - Demo of Conjunctive vs Disjunctive Explanations

This demo measures the differences in quality metrics and runtime of explanation generation between conjunctive and disjunctive explanations, on various datasets.\

In [35]:
import pandas as pd
import pd_explain
import time
import numpy as np
from sklearn.cluster import KMeans
separation_error = 0.8

## Adults dataset

In [36]:
adults = pd.read_csv(r"..\Examples\Datasets\adult.csv")
adults

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [37]:
adults_comparison_df = pd.DataFrame(columns=['Average Coverage', 'Average Separation Error', 'Runtime'],
                                    index=pd.MultiIndex(levels=[[], []], codes=[[], []], names=['Test', 'Explanation Type']))

### Full dataset test

In [38]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels='label', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
adults_comparison_df.loc[('Full dataset', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

Conjunctive explanation runtime: 1.2492327690124512


Coverage  \
Group / Cluster Explanation                                                   
<=50K           0 <= capital-gain <= 4243.5                            0.99   
                0.0 <= capital-gain <= 19999.8                          1.0   
                0 <= capital-gain <= 4668.5                            0.99   
                0 <= capital-gain <= 4668.5 AND 1 <= education-...     0.75   
                0.0 <= capital-gain <= 24999.75                         1.0   
                0.0 <= capital-gain <= 16666.5                          1.0   
                1 <= education-num <= 10                               0.76   
                0 <= capital-gain <= 4243.5 AND 1 <= education-...     0.75   
>50K            8.5 <= education-num <= 16.0                           0.97   
                marital-status == Married-civ-spouse                   0.85   
                marital-status == Married-civ-spouse AND 8.5 <=...     0.83   
                0 <= capital-gain <= 99999                              1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           0 <= capital-gain <= 4243.5                                     0.2  
                0.0 <= capital-gain <= 19999.8                                 0.23  
                0 <= capital-gain <= 4668.5                                     0.2  
                0 <= capital-gain <= 4668.5 AND 1 <= education-...             0.13  
                0.0 <= capital-gain <= 24999.75                                0.23  
                0.0 <= capital-gain <= 16666.5                                 0.23  
                1 <= education-num <= 10                                       0.15  
                0 <= capital-gain <= 4243.5 AND 1 <= education-...             0.13  
>50K            8.5 <= education-num <= 16.0                                   0.73  
                marital-status == Married-civ-spouse                           0.55  
                marital-status == Married-civ-spouse AND 8.5 <=...             0.51  
                0 <= capital-gain <= 99999                                     0.76

In [39]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels='label', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
adults_comparison_df.loc[('Full dataset', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

Disjunctive explanation runtime: 1.691772699356079


Coverage  \
Group / Cluster Explanation                                                   
<=50K           (0 <= capital-gain <= 4668.5)                          0.99   
                (0.0 <= capital-gain <= 19999.8)                        1.0   
                (6.0 <= education-num <= 11.0) OR (0 <= capital...      1.0   
                (0 <= capital-gain <= 4243.5)                          0.99   
                (6.0 <= education-num <= 11.0) OR (0 <= capital...      1.0   
                (0.0 <= capital-gain <= 24999.75)                       1.0   
                (1 <= education-num <= 10) OR (0 <= capital-gai...      1.0   
                (0.0 <= capital-gain <= 16666.5)                        1.0   
                (1 <= education-num <= 10) OR (0 <= capital-gai...      1.0   
                (1 <= education-num <= 10)                             0.76   
>50K            (8.5 <= education-num <= 16.0)                         0.97   
                marital-status == Married-civ-spouse                   0.85   
                (8.5 <= education-num <= 16.0) OR marital-statu...      1.0   
                (0 <= capital-gain <= 99999)                            1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           (0 <= capital-gain <= 4668.5)                                   0.2  
                (0.0 <= capital-gain <= 19999.8)                               0.23  
                (6.0 <= education-num <= 11.0) OR (0 <= capital...             0.22  
                (0 <= capital-gain <= 4243.5)                                   0.2  
                (6.0 <= education-num <= 11.0) OR (0 <= capital...             0.22  
                (0.0 <= capital-gain <= 24999.75)                              0.23  
                (1 <= education-num <= 10) OR (0 <= capital-gai...             0.22  
                (0.0 <= capital-gain <= 16666.5)                               0.23  
                (1 <= education-num <= 10) OR (0 <= capital-gai...             0.22  
                (1 <= education-num <= 10)                                     0.15  
>50K            (8.5 <= education-num <= 16.0)                                 0.73  
                marital-status == Married-civ-spouse                           0.55  
                (8.5 <= education-num <= 16.0) OR marital-statu...             0.74  
                (0 <= capital-gain <= 99999)                                   0.76

### Numeric only and categorical only tests

In [40]:
numeric_only = adults[['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]
categorical_only = adults[['workclass', 'education', 'marital-status', 'occupation', 'relationship']]
labels = adults['label']

In [41]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
adults_comparison_df.loc[('Numeric only', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

Conjunctive explanation runtime: 1.1632187366485596


Coverage  \
Group / Cluster Explanation                                                   
<=50K           0 <= capital-gain <= 4243.5                            0.99   
                0.0 <= capital-gain <= 19999.8                          1.0   
                0 <= capital-gain <= 4668.5                            0.99   
                0 <= capital-gain <= 4668.5 AND 1 <= education-...     0.75   
                0.0 <= capital-gain <= 24999.75                         1.0   
                0.0 <= capital-gain <= 16666.5                          1.0   
                1 <= education-num <= 10                               0.76   
                0 <= capital-gain <= 4243.5 AND 1 <= education-...     0.75   
>50K            8.5 <= education-num <= 16.0                           0.97   
                8.5 <= education-num <= 16.0 AND 29.5 <= age <= 90     0.91   
                29.5 <= age <= 90 AND 10 <= education-num <= 16        0.71   
                37 <= age <= 90                                        0.75   
                29.5 <= age <= 90                                      0.94   
                8.5 <= education-num <= 16.0 AND 37 <= age <= 90       0.72   
                0 <= capital-gain <= 99999                              1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           0 <= capital-gain <= 4243.5                                     0.2  
                0.0 <= capital-gain <= 19999.8                                 0.23  
                0 <= capital-gain <= 4668.5                                     0.2  
                0 <= capital-gain <= 4668.5 AND 1 <= education-...             0.13  
                0.0 <= capital-gain <= 24999.75                                0.23  
                0.0 <= capital-gain <= 16666.5                                 0.23  
                1 <= education-num <= 10                                       0.15  
                0 <= capital-gain <= 4243.5 AND 1 <= education-...             0.13  
>50K            8.5 <= education-num <= 16.0                                   0.73  
                8.5 <= education-num <= 16.0 AND 29.5 <= age <= 90             0.65  
                29.5 <= age <= 90 AND 10 <= education-num <= 16                0.56  
                37 <= age <= 90                                                0.65  
                29.5 <= age <= 90                                              0.68  
                8.5 <= education-num <= 16.0 AND 37 <= age <= 90               0.62  
                0 <= capital-gain <= 99999                                     0.76

In [42]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
adults_comparison_df.loc[('Numeric only', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

Disjunctive explanation runtime: 2.143446922302246


Coverage  \
Group / Cluster Explanation                                                   
<=50K           (0 <= capital-gain <= 4668.5)                          0.99   
                (0.0 <= capital-gain <= 19999.8)                        1.0   
                (6.0 <= education-num <= 11.0) OR (0 <= capital...      1.0   
                (0 <= capital-gain <= 4243.5)                          0.99   
                (6.0 <= education-num <= 11.0) OR (0 <= capital...      1.0   
                (0.0 <= capital-gain <= 24999.75)                       1.0   
                (1 <= education-num <= 10) OR (0 <= capital-gai...      1.0   
                (0.0 <= capital-gain <= 16666.5)                        1.0   
                (1 <= education-num <= 10) OR (0 <= capital-gai...      1.0   
                (1 <= education-num <= 10)                             0.76   
>50K            (37 <= age <= 90)                                      0.75   
                (8.5 <= education-num <= 16.0) OR (29.5 <= age ...      1.0   
                (8.5 <= education-num <= 16.0)                         0.97   
                (0 <= capital-gain <= 99999)                            1.0   
                (29.5 <= age <= 90)                                    0.94   
                (37 <= age <= 90) OR (8.5 <= education-num <= 1...     0.99   
                (29.5 <= age <= 90) OR (10 <= education-num <= 16)     0.98   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           (0 <= capital-gain <= 4668.5)                                   0.2  
                (0.0 <= capital-gain <= 19999.8)                               0.23  
                (6.0 <= education-num <= 11.0) OR (0 <= capital...             0.22  
                (0 <= capital-gain <= 4243.5)                                   0.2  
                (6.0 <= education-num <= 11.0) OR (0 <= capital...             0.22  
                (0.0 <= capital-gain <= 24999.75)                              0.23  
                (1 <= education-num <= 10) OR (0 <= capital-gai...             0.22  
                (0.0 <= capital-gain <= 16666.5)                               0.23  
                (1 <= education-num <= 10) OR (0 <= capital-gai...             0.22  
                (1 <= education-num <= 10)                                     0.15  
>50K            (37 <= age <= 90)                                              0.65  
                (8.5 <= education-num <= 16.0) OR (29.5 <= age ...             0.75  
                (8.5 <= education-num <= 16.0)                                 0.73  
                (0 <= capital-gain <= 99999)                                   0.76  
                (29.5 <= age <= 90)                                            0.68  
                (37 <= age <= 90) OR (8.5 <= education-num <= 1...             0.74  
                (29.5 <= age <= 90) OR (10 <= education-num <= 16)             0.73

In [43]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
adults_comparison_df.loc[('Categorical only', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

Conjunctive explanation runtime: 0.47734808921813965


Coverage  \
Group / Cluster Explanation                                                   
<=50K           occupation != Prof-specialty AND education != B...     0.82   
                occupation != Prof-specialty                           0.91   
>50K            marital-status == Married-civ-spouse                   0.85   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           occupation != Prof-specialty AND education != B...             0.18  
                occupation != Prof-specialty                                   0.21  
>50K            marital-status == Married-civ-spouse                           0.55

In [44]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
adults_comparison_df.loc[('Categorical only', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

Disjunctive explanation runtime: 0.6061787605285645


Coverage  \
Group / Cluster Explanation                                                   
<=50K           occupation != Prof-specialty OR education != Ba...     0.96   
                occupation != Prof-specialty                           0.91   
>50K            marital-status == Married-civ-spouse                   0.85   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           occupation != Prof-specialty OR education != Ba...             0.23  
                occupation != Prof-specialty                                   0.21  
>50K            marital-status == Married-civ-spouse                           0.55

### Groupby tests

In [45]:
single_attribute_groupby = adults.groupby('relationship').mean()
multi_attribute_groupby = adults.groupby(['sex', 'label']).mean()

In [46]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
adults_comparison_df.loc[('Single attribute groupby', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

Conjunctive explanation runtime: 2.243629217147827


Coverage  \
Group / Cluster Explanation                                                   
Not-in-family   marital-status != Married-civ-spouse                    1.0   
                marital-status != Married-civ-spouse AND 25.5 <...     0.81   
                marital-status != Married-civ-spouse AND 19.5 <...     0.98   
                marital-status != Married-civ-spouse AND 22.5 <...     0.91   
Husband         marital-status == Married-civ-spouse AND sex !=...      1.0   
                marital-status == Married-civ-spouse AND sex ==...      1.0   
                marital-status == Married-civ-spouse                    1.0   
Wife            marital-status == Married-civ-spouse AND sex ==...     0.99   
                marital-status == Married-civ-spouse AND sex !=...     0.99   
Own-child       marital-status != Married-civ-spouse AND 17.0 <...     0.79   
                marital-status != Married-civ-spouse                   0.98   
                17.0 <= age <= 29.166666666666664 AND marital-s...     0.76   
                17.0 <= age <= 35.25 AND marital-status == Neve...     0.83   
                17 <= age <= 37 AND marital-status == Never-mar...     0.85   
                marital-status != Married-civ-spouse AND 17 <= ...     0.71   
                17.0 <= age <= 35.25                                    0.9   
                17 <= age <= 26                                        0.72   
                17.0 <= age <= 53.5 AND marital-status == Never...     0.89   
                17.0 <= age <= 41.33333333333333 AND marital-st...     0.87   
                marital-status == Never-married                        0.89   
                17 <= age <= 31 AND marital-status == Never-mar...     0.79   
                17 <= age <= 28 AND marital-status == Never-mar...     0.75   
                marital-status != Married-civ-spouse AND 17 <= ...     0.77   
                17 <= age <= 28                                        0.78   
                marital-status != Married-civ-spouse AND 17 <= ...      0.9   
                17.0 <= age <= 31.6 AND marital-status == Never...     0.79   
                marital-status != Married-civ-spouse AND 17.0 <...     0.93   
                marital-status != Married-civ-spouse AND 17.0 <...     0.97   
Unmarried       marital-status != Married-civ-spouse AND marita...     0.74   
                marital-status != Married-civ-spouse AND 25.5 <...      0.9   
                marital-status != Married-civ-spouse AND 27.5 <...     0.86   
                marital-status != Married-civ-spouse AND 22.5 <...     0.96   
Other-relative  No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Not-in-family   marital-status != Married-civ-spouse                           0.53  
                marital-status != Married-civ-spouse AND 25.5 <...             0.43  
                marital-status != Married-civ-spouse AND 19.5 <...             0.49  
                marital-status != Married-civ-spouse AND 22.5 <...             0.45  
Husband         marital-status == Married-civ-spouse AND sex !=...             0.01  
                marital-status == Married-civ-spouse AND sex ==...             0.01  
                marital-status == Married-civ-spouse                           0.12  
Wife            marital-status == Married-civ-spouse AND sex ==...             0.07  
                marital-status == Married-civ-spouse AND sex !=...             0.07  
Own-child       marital-status != Married-civ-spouse AND 17.0 <...             0.49  
                marital-status != Married-civ-spouse                           0.72  
                17.0 <= age <= 29.166666666666664 AND marital-s...             0.46  
                17.0 <= age <= 35.25 AND marital-status == Neve...             0.52  
           

In [47]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
adults_comparison_df.loc[('Single attribute groupby', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

Disjunctive explanation runtime: 2.6881487369537354


Coverage  \
Group / Cluster Explanation                                                   
Not-in-family   marital-status != Married-civ-spouse                    1.0   
Husband         marital-status == Married-civ-spouse                    1.0   
Own-child       marital-status != Married-civ-spouse OR (17.0 <...      1.0   
                (17.0 <= age <= 35.25)                                  0.9   
                (17.0 <= age <= 31.6) OR marital-status == Neve...     0.94   
                (17 <= age <= 28)                                      0.78   
                marital-status != Married-civ-spouse                   0.98   
                (17.0 <= age <= 35.25) OR marital-status == Nev...     0.95   
                (17 <= age <= 28) OR marital-status == Never-ma...     0.93   
                marital-status == Never-married                        0.89   
                marital-status != Married-civ-spouse OR (17 <= ...     0.99   
                (17 <= age <= 26)                                      0.72   
                (17 <= age <= 31) OR marital-status == Never-ma...     0.94   
Other-relative  No explanation found                                    NaN   
Unmarried       No explanation found                                    NaN   
Wife            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Not-in-family   marital-status != Married-civ-spouse                           0.53  
Husband         marital-status == Married-civ-spouse                           0.12  
Own-child       marital-status != Married-civ-spouse OR (17.0 <...             0.77  
                (17.0 <= age <= 35.25)                                          0.7  
                (17.0 <= age <= 31.6) OR marital-status == Neve...             0.67  
                (17 <= age <= 28)                                              0.55  
                marital-status != Married-civ-spouse                           0.72  
                (17.0 <= age <= 35.25) OR marital-status == Nev...             0.71  
                (17 <= age <= 28) OR marital-status == Never-ma...             0.63  
                marital-status == Never-married                                0.58  
                marital-status != Married-civ-spouse OR (17 <= ...             0.73  
                (17 <= age <= 26)                                              0.49  
                (17 <= age <= 31) OR marital-status == Never-ma...             0.67  
Other-relative  No explanation found                                            NaN  
Unmarried       No explanation found                                            NaN  
Wife            No explanation found                                            NaN

In [48]:
start_time = time.time()
conj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
adults_comparison_df.loc[('Multi attribute groupby', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

Conjunctive explanation runtime: 1.185699462890625


Coverage  \
Group / Cluster     Explanation                                                   
('Male', '<=50K')   0.0 <= capital-gain <= 19999.8                          1.0   
                    0 <= capital-gain <= 4932.5                            0.99   
                    0 <= capital-gain <= 4668.5                            0.99   
                    relationship != infrequent_sklearn AND 0 <= cap...     0.96   
                    relationship != infrequent_sklearn AND 0 <= cap...     0.96   
                    0.0 <= capital-gain <= 24999.75                         1.0   
                    0.0 <= capital-gain <= 16666.5                          1.0   
                    0.0 <= capital-gain <= 49999.5                          1.0   
                    0.0 <= capital-gain <= 33333.0                          1.0   
                    0 <= capital-gain <= 99999                              1.0   
('Female', '<=50K') relationship != Husband AND 0 <= capital-gain <...     0.99   
                    relationship != Husband AND 0 <= capital-gain <...     0.99   
                    relationship != Husband                                 1.0   
('Male', '>50K')    relationship != infrequent_sklearn                      1.0   
                    relationship == Husband                                0.89   
('Female', '>50K')  No explanation found                                    NaN   

                                                                       Separation Error  
Group / Cluster     Explanation                                                          
('Male', '<=50K')   0.0 <= capital-gain <= 19999.8                                 0.53  
                    0 <= capital-gain <= 4932.5                                    0.51  
                    0 <= capital-gain <= 4668.5                                    0.51  
                    relationship != infrequent_sklearn AND 0 <= cap...             0.49  
                    relationship != infrequent_sklearn AND 0 <= cap...             0.49  
                    0.0 <= capital-gain <= 24999.75                                0.53  
                    0.0 <= capital-gain <= 16666.5                                 0.53  
                    0.0 <= capital-gain <= 49999.5                                 0.53  
                    0.0 <= capital-gain <= 33333.0                                 0.53  
                    0 <= capital-gain <= 99999                                     0.53  
('Female', '<=50K') relationship != Husband AND 0 <= capital-gain <...             0.49  
                    relationship != Husband AND 0 <= capital-gain <...             0.49  
                    relationship != Husband                                         0.5  
('Male', '>50K')    relationship != infrequent_sklearn                             0.78  
                    relationship == Husband                                        0.55  
('Female', '>50K')  No explanation found                                            NaN

In [49]:
start_time = time.time()
disj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
adults_comparison_df.loc[('Multi attribute groupby', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

Disjunctive explanation runtime: 1.568009376525879


Coverage  \
Group / Cluster     Explanation                                   
('Male', '<=50K')   (0 <= capital-gain <= 4668.5)          0.99   
                    (0.0 <= capital-gain <= 19999.8)        1.0   
                    (0 <= capital-gain <= 4932.5)          0.99   
                    (0.0 <= capital-gain <= 49999.5)        1.0   
                    (0.0 <= capital-gain <= 24999.75)       1.0   
                    (0 <= capital-gain <= 99999)            1.0   
                    (0.0 <= capital-gain <= 16666.5)        1.0   
                    (0.0 <= capital-gain <= 33333.0)        1.0   
('Female', '<=50K') relationship != Husband                 1.0   
('Male', '>50K')    relationship != infrequent_sklearn      1.0   
                    relationship == Husband                0.89   
('Female', '>50K')  No explanation found                    NaN   

                                                       Separation Error  
Group / Cluster     Explanation                                          
('Male', '<=50K')   (0 <= capital-gain <= 4668.5)                  0.51  
                    (0.0 <= capital-gain <= 19999.8)               0.53  
                    (0 <= capital-gain <= 4932.5)                  0.51  
                    (0.0 <= capital-gain <= 49999.5)               0.53  
                    (0.0 <= capital-gain <= 24999.75)              0.53  
                    (0 <= capital-gain <= 99999)                   0.53  
                    (0.0 <= capital-gain <= 16666.5)               0.53  
                    (0.0 <= capital-gain <= 33333.0)               0.53  
('Female', '<=50K') relationship != Husband                         0.5  
('Male', '>50K')    relationship != infrequent_sklearn             0.78  
                    relationship == Husband                        0.55  
('Female', '>50K')  No explanation found                            NaN

### Clustering test

#### 3 clusters

In [50]:
cluster_labels = KMeans(n_clusters=3).fit_predict(pd.get_dummies(adults))

In [51]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
adults_comparison_df.loc[('3 clusters', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

Conjunctive explanation runtime: 2.1946029663085938


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,149630.5 <= fnlwgt <= 291625.5,1.0,0.0
1,12285 <= fnlwgt <= 149630.5,1.0,0.0
2,291625.5 <= fnlwgt <= 1490400,1.0,0.0


In [52]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
adults_comparison_df.loc[('3 clusters', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

Disjunctive explanation runtime: 3.3409926891326904


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,(149630.5 <= fnlwgt <= 291625.5),1.0,0.0
1,(12285 <= fnlwgt <= 149630.5),1.0,0.0
2,(291625.5 <= fnlwgt <= 1490400),1.0,0.0


#### 15 clusters

In [53]:
cluster_labels = KMeans(n_clusters=15).fit_predict(pd.get_dummies(adults))

In [54]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
adults_comparison_df.loc[('15 clusters', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

Conjunctive explanation runtime: 11.032832860946655


Coverage  \
Group / Cluster Explanation                                                   
0               58263.5 <= fnlwgt <= 93908.0                            1.0   
1               281946.5 <= fnlwgt <= 1490400                           1.0   
2               178147 <= fnlwgt <= 237646                              1.0   
                178147 <= fnlwgt <= 210736                             0.96   
3               433247.5 <= fnlwgt <= 1490400 AND 17.0 <= age <...     0.74   
                433247.5 <= fnlwgt <= 1490400                           1.0   
                433247.5 <= fnlwgt <= 1490400 AND 17.0 <= age <...      0.9   
4               673088.0 <= fnlwgt <= 1490400                           1.0   
5               210736 <= fnlwgt <= 279173                              1.0   
                210736 <= fnlwgt <= 279173 AND occupation != Pr...     0.89   
6               93908.0 <= fnlwgt <= 126201.5                           1.0   
7               12285 <= fnlwgt <= 58263.5                              1.0   
8               371701.5 <= fnlwgt <= 1490400 AND 8.5 <= educat...     0.83   
                371701.5 <= fnlwgt <= 1490400                           1.0   
9               962971.0 <= fnlwgt <= 1490400                           1.0   
10              210736 <= fnlwgt <= 279173                             0.94   
                244788.0 <= fnlwgt <= 1490400                           1.0   
11              524879.0 <= fnlwgt <= 1490400                           1.0   
                524879.0 <= fnlwgt <= 1490400 AND 8.5 <= educat...     0.83   
12              126201.5 <= fnlwgt <= 156447.0                          1.0   
13              323521.0 <= fnlwgt <= 1490400 AND 28.5 <= age <...     0.74   
                323521.0 <= fnlwgt <= 1490400                           1.0   
14              156447.0 <= fnlwgt <= 184008.0                          1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               58263.5 <= fnlwgt <= 93908.0                                    0.0  
1               281946.5 <= fnlwgt <= 1490400                                  0.65  
2               178147 <= fnlwgt <= 237646                                     0.43  
                178147 <= fnlwgt <= 210736                                     0.18  
3               433247.5 <= fnlwgt <= 1490400 AND 17.0 <= age <...             0.34  
                433247.5 <= fnlwgt <= 1490400                                  0.36  
                433247.5 <= fnlwgt <= 1490400 AND 17.0 <= age <...             0.35  
4               673088.0 <= fnlwgt <= 1490400                                  0.16  
5               210736 <= fnlwgt <= 279173                                     0.43  
                210736 <= fnlwgt <= 279173 AND occupation != Pr...             0.42  
6               93908.0 <= fnlwgt <= 126201.5                                   0.0  
7               12285 <= fnlwgt <= 58263.5                                      0.0  
8               371701.5 <= fnlwgt <= 1490400 AND 8.5 <= educat...             0.43  
                371701.5 <= fnlwgt <= 1490400                                  0.44  
9               962971.0 <= fnlwgt <= 1490400                                   0.0  
10              210736 <= fnlwgt <= 279173                                     0.61  
                244788.0 <= fnlwgt <= 1490400                                   0.7  
11              524879.0 <= fnlwgt <= 1490400                                  0.29  
                524879.0 <= fnlwgt <= 1490400 AND 8.5 <= educat...             0.28  
12              126201.5 <= fnlwgt <= 156447.0                                  0.0  
13              323521.0 <= fnlwgt <= 1490400 AND 28.5 <= age <...              0.5  
                323521.0 <= fnlwgt <= 1490400                                  0.53  
14              156447.0 <=

In [55]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
adults_comparison_df.loc[('15 clusters', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

Disjunctive explanation runtime: 10.4983069896698


Coverage Separation Error
Group / Cluster Explanation                                               
0               (58263.5 <= fnlwgt <= 93908.0)        1.0              0.0
1               (281946.5 <= fnlwgt <= 1490400)       1.0             0.65
2               (178147 <= fnlwgt <= 210736)         0.96             0.18
                (178147 <= fnlwgt <= 237646)          1.0             0.43
3               (433247.5 <= fnlwgt <= 1490400)       1.0             0.36
4               (673088.0 <= fnlwgt <= 1490400)       1.0             0.16
5               (210736 <= fnlwgt <= 279173)          1.0             0.43
6               (93908.0 <= fnlwgt <= 126201.5)       1.0              0.0
7               (12285 <= fnlwgt <= 58263.5)          1.0              0.0
8               (371701.5 <= fnlwgt <= 1490400)       1.0             0.44
9               (962971.0 <= fnlwgt <= 1490400)       1.0              0.0
10              (244788.0 <= fnlwgt <= 1490400)       1.0              0.7
                (210736 <= fnlwgt <= 279173)         0.94             0.61
11              (524879.0 <= fnlwgt <= 1490400)       1.0             0.29
12              (126201.5 <= fnlwgt <= 156447.0)      1.0              0.0
13              (323521.0 <= fnlwgt <= 1490400)       1.0             0.53
14              (156447.0 <= fnlwgt <= 184008.0)      1.0              0.0

### Summary of results

In [56]:
adults_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Full dataset             conjunctive                0.9075   
                         disjunctive              0.968571   
Numeric only             conjunctive              0.882667   
                         disjunctive              0.962941   
Categorical only         conjunctive                  0.86   
                         disjunctive              0.906667   
Single attribute groupby conjunctive              0.875312   
                         disjunctive              0.924615   
Multi attribute groupby  conjunctive              0.984667   
                         disjunctive              0.988182   
3 clusters               conjunctive                   1.0   
                         disjunctive                   1.0   
15 clusters              conjunctive               0.94913   
                         disjunctive              0.994118   

                                          Average Separation Error    Runtime  
Test                     Explanation Type                                      
Full dataset             conjunctive                        0.3375   1.249233  
                         disjunctive                          0.35   1.691773  
Numeric only             conjunctive                          0.41   1.163219  
                         disjunctive                      0.421176   2.143447  
Categorical only         conjunctive                      0.313333   0.477348  
                         disjunctive                          0.33   0.606179  
Single attribute groupby conjunctive                      0.482813   2.243629  
                         disjunctive                      0.605385   2.688149  
Multi attribute groupby  conjunctive                      0.532667   1.185699  
                         disjunctive                      0.548182   1.568009  
3 clusters               conjunctive                           0.0   2.194603  
                         disjunctive                           0.0   3.340993  
15 clusters              conjunctive                      0.308696  11.032833  
                         disjunctive                      0.281176  10.498307

## Spotify dataset

In [57]:
spotify = pd.read_csv(r"..\Examples\Datasets\spotify_all.csv")
spotify

,acousticness,artists,danceability,energy,explicit,id,instrumentalness,key,liveness,loudness,...,name,popularity,speechiness,tempo,valence,year,decade,popularity_score,main_artist,duration_minutes
0,0.991000,['Mamie Smith'],0.598,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,...,Keep A Song In Your Soul,12,0.0936,149.976,0.6340,1920,1920,10,Mamie Smith,2.805550
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,...,I Put A Spell On You,7,0.0534,86.889,0.9500,1920,1920,0,Screamin' Jay Hawkins,2.503333
2,0.993000,['Mamie Smith'],0.647,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,...,Golfing Papa,4,0.1740,97.600,0.6890,1920,1920,0,Mamie Smith,2.730450
3,0.000173,['Oscar Velazquez'],0.730,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,...,True House Music - Xavier Santos & Carlos Gomi...,17,0.0425,127.997,0.0422,1920,1920,10,Oscar Velazquez,7.034783
4,0.295000,['Mixe'],0.704,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,...,Xuniverxe,2,0.0768,122.076,0.2990,1920,1920,0,Mixe,2.753733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,...,The One,0,0.0356,125.972,0.1860,2020,2020,0,DJ Combo,2.460250
174385,0.795000,['Alessia Cara'],0.429,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,...,A Little More,0,0.0360,94.710,0.2280,2021,2020,0,Alessia Cara,2.412000
174386,0.806000,['Roger Fly'],0.671,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,...,Together,0,0.0282,108.058,0.7140,2020,2020,0,Roger Fly,3.635783
174387,0.920000,['Taylor Swift'],0.462,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,...,champagne problems,69,0.0377,171.319,0.3200,2021,2020,60,Taylor Swift,4.066667


In [58]:
spotify_comparison_df = pd.DataFrame(columns=['Average Coverage', 'Average Separation Error', 'Runtime'],
                                    index=pd.MultiIndex(levels=[[], []], codes=[[], []], names=['Test', 'Explanation Type']))

### Full dataset test

In [59]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels='key', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
spotify_comparison_df.loc[('Full dataset', 'conjunctive'), :] = [np.mean(conj_res['Coverage']), np.mean(conj_res['Separation Error']), conj_time]
conj_res

KeyboardInterrupt: 

In [34]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels='key', explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
spotify_comparison_df.loc[('Full dataset', 'disjunctive'), :] = [np.mean(disj_res['Coverage']), np.mean(disj_res['Separation Error']), disj_time]
disj_res

KeyboardInterrupt: 

In [83]:
spotify_comparison_df

Average Coverage Average Separation Error  \
Test         Explanation Type                                             
Full dataset conjunctive                   NaN                      NaN   
             disjunctive                   NaN                      NaN   

                                 Runtime  
Test         Explanation Type             
Full dataset conjunctive       28.525063  
             disjunctive       39.150858

### Numeric only and categorical only tests

In [ ]:
numeric_only = spotify[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]
categorical_only = spotify[['name', 'main_artist', 'year']]
labels = spotify['decade']